In [5]:
# Set Variables
WKDIR=/workspace/hraijc/HiC_trials/


In [ ]:
READ1=/workspace/hraijc/10p_hic_1.fastq.gz
READ2=/workspace/hraijc/10p_hic_2.fastq.gz
ASSEMBLY=/workspace/hraijc/priGClenfilt.fasta
TEMP=/workspace/hraijc/TEMP/
APREFIX=2n_pri

In [6]:
cd ${WKDIR}
mkdir -p ${WKDIR}/log

In [7]:

sbatch << EOF
#!/bin/bash
#SBATCH -J hic
#SBATCH -o ${LOGDIR}/%J.out
#SBATCH -e ${LOGDIR}/%J.err
#SBATCH --cpus-per-task=12
#SBATCH --mem-per-cpu=16G
#SBATCH --time=88:00:00

module load bwa/0.7.18
module load samtools/1.20


cd ${WKDIR}

### Align reads with BWA. Use -5SP for Hi-C reads.#############################
#samtools faidx ${ASSEMBLY}
#bwa index ${ASSEMBLY}
#bwa mem -5SP -t12 ${ASSEMBLY} ${READ1} ${READ2} -o ${TEMP}/${APREFIX}.sam


### Flag PCR Duplicates with SAMBLASTER #######################################
/workspace/hraijc/git_clones/samblaster/samblaster -i ${TEMP}/${APREFIX}.sam -o ${TEMP}/${APREFIX}_marked_byread.sam


### Remove unmmaped and non-primary aligned reads. Sort and Index bam files.###
samtools view -S -b -h -@ 12 -F 2316 ${TEMP}/${APREFIX}_marked_byread.sam > ${TEMP}/${APREFIX}_presort_marked.bam
samtools sort -@ 12 ${TEMP}/${APREFIX}_presort_marked.bam -o ${APREFIX}.bam

### Run YAHS with contig and scaffolding error correction. ####################
/workspace/hraijc/git_clones/yahs/yahs --no-contig-ec --no-mem-check --telo-motif CCCTAAA ${ASSEMBLY} ${APREFIX}.bam -o ${APREFIX}_yahs1

EOF


In [ ]:
### Make heatmap of YAHS output ###############################################
cd ${WKDIR}

out=${APREFIX}_yahs1 # prefix of outfiles produced by YAHS.  
contigs=${ASSEMBLY} # need to be indexed, i.e., test.fa.gz.fai in same directory
wkdir=${WKDIR}


sbatch --job-name=hic_mapyahs \
    -o ${WKDIR}/log/%J.out \
    -e ${WKDIR}/log/%J.err \
    --cpus-per-task=8 \
    --mem=24G \
    --time=03:10:00 \
    --export=out=$out,contigs=$contigs,wkdir=$wkdir \
    /workspace/hraijc/Gitrepos/High-quality-genomes/Methods/DNase_HiC/notebooks/yahs_contactmapgen2.sh